# Data preparation


- Join CSV files
- brief overview about the data so it can be cleaned and preprocessed
- Cleaning and preprocessing
    - rename columns 
    - convert datatypes
    - Duplicates
    - Data Aggreagation
    - impute missing values


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split
import pandas as pd
import os


## Merging CSVs
In this chapter we join the CSV files that have data about car accidents in germany in a time span from 2016 to 2022. The source of the data is [unfallatlas.statistikportal](https://unfallatlas.statistikportal.de/)

In [2]:

#merge Traffic Accident Data from 2016 to 2022
folder_path = "C:\Projekte\TDS\TDS2324-TrafficAccidents\Data\TrafficAccidentData"
file_list = os.listdir(folder_path)
merged_data = pd.DataFrame()

# Iterate over each file in the folder and add to merged_data
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    data = pd.read_csv(file_path,sep=';')
    merged_data = pd.concat([merged_data, data], ignore_index=True)

print(merged_data.columns)

C:\Users\miche\AppData\Local\Temp\ipykernel_26408\3181226627.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path,sep=';')
C:\Users\miche\AppData\Local\Temp\ipykernel_26408\3181226627.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path,sep=';')


Index(['FID', 'OBJECTID', 'ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR',
       'UMONAT', 'USTUNDE', 'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1',
       'ULICHTVERH', 'STRZUSTAND', 'IstRad', 'IstPKW', 'IstFuss', 'IstKrad',
       'IstGkfz', 'IstSonstige', 'LINREFX', 'LINREFY', 'XGCSWGS84',
       'YGCSWGS84', 'UIDENTSTLA', 'OBJECTID_1', 'UIDENTSTLAE'],
      dtype='object')


## Introducing the data set

To get a first impression of the dataset before diving into the data cleaning and preprocessing stages, we conduct an initial overview. This step is vital for grasping the basic structure, size, and characteristics of the data.


In [3]:
merged_data.head(10)

,FID,OBJECTID,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstKrad,IstGkfz,IstSonstige,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,UIDENTSTLA,OBJECTID_1,UIDENTSTLAE
0,0.0,1.0,1,0,53,120,2016,1,9,5,...,0,0.0,0,"606982,393999999970000","5954659,924999999800000","10,621659329000000","53,729614888000000",NaN,NaN,NaN
1,1.0,2.0,1,0,57,10,2016,1,17,3,...,0,0.0,0,"574882,533000000050000","6011440,850999999800000","10,149175783000000","54,245452583999999",NaN,NaN,NaN
2,2.0,3.0,1,0,62,8,2016,1,0,5,...,0,0.0,0,"599934,687500000000000","5964608,596199999600000","10,518094344000000","53,820402504999997",NaN,NaN,NaN
3,3.0,4.0,1,0,3,0,2016,1,15,5,...,0,0.0,1,"610709,348699999970000","5968284,242300000000000","10,683020702000000","53,851243101000001",NaN,NaN,NaN
4,4.0,5.0,1,0,55,28,2016,1,14,1,...,0,0.0,0,"605690,790400000060000","6009152,214700000400000","10,620986018000000","54,219458582000001",NaN,NaN,NaN
5,5.0,6.0,1,0,61,44,2016,1,17,6,...,0,0.0,0,"542547,328499999950000","5963921,747000000400000","9,646338030000001","53,822093844999998",NaN,NaN,NaN
6,6.0,7.0,1,0,60,54,2016,1,17,3,...,0,0.0,0,"558412,407499999970000","5969889,909599999900000","9,888452573000000","53,874189672000000",NaN,NaN,NaN
7,7.0,8.0,1,0,55,29,2016,1,11,1,...,0,0.0,0,"623911,704400000050000","6008211,261900000300000","10,899895740000000","54,206924608999998",NaN,NaN,NaN
8,8.0,9.0,1,0,60,85,2016,1,16,2,...,0,0.0,0,"579664,308100000020000","5965471,277499999900000","10,210478533000000","53,831655902000001",NaN,NaN,NaN
9,9.0,10.0,1,0,54,126,2016,1,10,4,...,0,0.0,1,"494320,527400000020000","6066809,662200000100000","8,911765602999999","54,748506792999997",NaN,NaN,NaN


In [4]:
merged_data.shape

(1554834, 28)

In [5]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554834 entries, 0 to 1554833
Data columns (total 28 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   FID          151673 non-null   float64
 1   OBJECTID     1342966 non-null  float64
 2   ULAND        1554834 non-null  int64  
 3   UREGBEZ      1554834 non-null  int64  
 4   UKREIS       1554834 non-null  int64  
 5   UGEMEINDE    1554834 non-null  int64  
 6   UJAHR        1554834 non-null  int64  
 7   UMONAT       1554834 non-null  int64  
 8   USTUNDE      1554834 non-null  int64  
 9   UWOCHENTAG   1554834 non-null  int64  
 10  UKATEGORIE   1554834 non-null  int64  
 11  UART         1554834 non-null  int64  
 12  UTYP1        1554834 non-null  int64  
 13  ULICHTVERH   1554834 non-null  int64  
 14  STRZUSTAND   1554834 non-null  int64  
 15  IstRad       1554834 non-null  int64  
 16  IstPKW       1554834 non-null  int64  
 17  IstFuss      1554834 non-null  int64  
 18  Is

## Cleaning and preprocessing
In this section, we undertake a series of steps crucial for refining and preparing our dataset for analysis. These steps are designed to enhance the quality and usability of the data:

- **Drop Data:** Remove unnecessary features.
- **Convert Data:** Transform data into suitable formats.
- **Check Duplicates:** Eliminate duplicate entries.
- **Rename Features:** Standardize feature names for clarity.
- **Aggregate Data:** Combine similar features for simplicity.
- **Handle Missing Values:** Impute missing data to maintain dataset integrity.


### Drop Data
We are removing features that are unnecessary for this project. This includes elements that are primarily foreign keys or IDs, which are either irrelevant or inconsistently named across the various datasets that were merged. Additionally, we are discarding the 'linref' feature, as we have chosen to utilize the WGS84 coordinate reference system instead

In [6]:
# drop columns that are not needed
merged_data.drop(['FID', 'OBJECTID', 'OBJECTID_1', 'UIDENTSTLA', 'UIDENTSTLAE','LINREFX','LINREFY'], axis=1, inplace=True)
print(merged_data.columns)

Index(['ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR', 'UMONAT', 'USTUNDE',
       'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'STRZUSTAND',
       'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige',
       'XGCSWGS84', 'YGCSWGS84'],
      dtype='object')


### Converting Datatypes
Since the coordinates are saved as strings we  have to convert them in a datatype we can later work with. 

In [7]:
# convert coordinates to float
merged_data['XGCSWGS84'] = merged_data['XGCSWGS84'].astype(str).str.replace(',', '.').astype(float)
merged_data['YGCSWGS84'] = merged_data['YGCSWGS84'].astype(str).str.replace(',', '.').astype(float)

merged_data.head(10)


,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,...,ULICHTVERH,STRZUSTAND,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,XGCSWGS84,YGCSWGS84
0,1,0,53,120,2016,1,9,5,2,8,...,0,2,0,1,0,0,0.0,0,10.621659,53.729615
1,1,0,57,10,2016,1,17,3,3,1,...,2,1,0,1,0,0,0.0,0,10.149176,54.245453
2,1,0,62,8,2016,1,0,5,3,9,...,2,0,0,1,0,0,0.0,0,10.518094,53.820403
3,1,0,3,0,2016,1,15,5,3,5,...,0,0,1,0,0,0,0.0,1,10.683021,53.851243
4,1,0,55,28,2016,1,14,1,3,8,...,0,2,0,1,0,0,0.0,0,10.620986,54.219459
5,1,0,61,44,2016,1,17,6,3,9,...,2,1,0,1,0,0,0.0,0,9.646338,53.822094
6,1,0,60,54,2016,1,17,3,3,6,...,2,0,1,0,1,0,0.0,0,9.888453,53.874190
7,1,0,55,29,2016,1,11,1,2,2,...,0,1,0,1,0,0,0.0,0,10.899896,54.206925
8,1,0,60,85,2016,1,16,2,2,4,...,0,1,0,1,0,0,0.0,0,10.210479,53.831656
9,1,0,54,126,2016,1,10,4,2,5,...,0,0,0,1,0,0,0.0,1,8.911766,54.748507


### Checking for Duplicates
In this section of the notebook, we create a temporary DataFrame 'temp_df' by making a copy of 'merged_data'. This ensures that any modifications do not affect the original dataset. We then use the *drop_duplicates*() method on 'temp_df' to remove any duplicate rows, enhancing the data's uniqueness and integrity.

In [8]:
# creating a copy of merged_data and dropping duplicates
temp_df = merged_data.copy()
temp_df.drop_duplicates(inplace=True)

# number of rows for comparison
num_rows_merged_data = merged_data.shape[0]
num_rows_temp_df = temp_df.shape[0]

print("number of rows in merged_data: ", num_rows_merged_data)
print("number of rows in temp_df: ", num_rows_temp_df)    
print("difference in rows: ", num_rows_temp_df - num_rows_merged_data)

number of rows in merged_data:  1554834
number of rows in temp_df:  1554694
difference in rows:  -140


Since there is a difference of 140 rows between the 'merged_data' and its copy after the drop of duplicates we know that there are 140 duplicates in our dataset which will be further examined:

In [9]:
# show duplicates 
duplicates = merged_data[merged_data.duplicated(keep=False)]
duplicates.head(14)

,ULAND,UREGBEZ,UKREIS,UGEMEINDE,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,UKATEGORIE,UART,...,ULICHTVERH,STRZUSTAND,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,XGCSWGS84,YGCSWGS84
52335,7,3,39,43,2016,3,13,4,3,1,...,0,0,0,1,0,0,0.0,0,8.355149,49.862495
52339,7,3,39,43,2016,3,13,4,3,1,...,0,0,0,1,0,0,0.0,0,8.355149,49.862495
54140,7,1,37,69,2016,6,5,5,3,2,...,0,0,0,1,0,0,0.0,0,7.258458,50.388285
54478,7,1,37,69,2016,6,5,5,3,2,...,0,0,0,1,0,0,0.0,0,7.258458,50.388285
58019,7,1,43,59,2016,9,14,5,3,2,...,0,0,0,1,0,0,0.0,0,7.730177,50.502286
58247,7,1,43,59,2016,9,14,5,3,2,...,0,0,0,1,0,0,0.0,0,7.730177,50.502286
58314,7,3,12,0,2016,10,21,7,3,9,...,2,1,0,1,0,1,0.0,0,7.715006,49.448939
58484,7,3,12,0,2016,10,21,7,3,9,...,2,1,0,1,0,1,0.0,0,7.715006,49.448939
60015,7,3,38,6,2016,12,19,1,3,2,...,2,0,0,1,0,0,0.0,0,8.340503,49.413595
60256,7,3,38,6,2016,12,19,1,3,2,...,2,0,0,1,0,0,0.0,0,8.340503,49.413595


The table shows that there are indeed duplicates in the dataset. Since the coordinates are exremly detailed, it is very unlilkely that the accidents happend the same time with the same participants at the exact same spot. After doublechecking the accidents on the map we decided to drop them.

In [10]:
#drop duplicates
merged_data.drop_duplicates(inplace=True)
num_rows_merged_data = merged_data.shape[0]

print("number of rows in merged_data: ", num_rows_merged_data)
print("number of rows in temp_df: ", num_rows_temp_df)
print("difference in rows: ", num_rows_temp_df - num_rows_merged_data)


number of rows in merged_data:  1554694
number of rows in temp_df:  1554694
difference in rows:  0


Since they have now the same amount of rows, there are no further duplicates in the dataset.


### Renaming Features
In this section, we address the issue of feature names in our dataset being formatted in a way that hinders efficient data handling. The objective is to modify these names to make them more intuitive and consistent, facilitating easier manipulation and analysis.

In [11]:
merged_data.columns

Index(['ULAND', 'UREGBEZ', 'UKREIS', 'UGEMEINDE', 'UJAHR', 'UMONAT', 'USTUNDE',
       'UWOCHENTAG', 'UKATEGORIE', 'UART', 'UTYP1', 'ULICHTVERH', 'STRZUSTAND',
       'IstRad', 'IstPKW', 'IstFuss', 'IstKrad', 'IstGkfz', 'IstSonstige',
       'XGCSWGS84', 'YGCSWGS84'],
      dtype='object')

In [12]:
# cols in lower case   
merged_data.columns =  [col.lower() for col in merged_data.columns]
merged_data.columns



Index(['uland', 'uregbez', 'ukreis', 'ugemeinde', 'ujahr', 'umonat', 'ustunde',
       'uwochentag', 'ukategorie', 'uart', 'utyp1', 'ulichtverh', 'strzustand',
       'istrad', 'istpkw', 'istfuss', 'istkrad', 'istgkfz', 'istsonstige',
       'xgcswgs84', 'ygcswgs84'],
      dtype='object')

Since the U in the colnames stands for 'Unfall' and its obvious that we are having a dataset with data of accidents the U will be deleted to make things clearer. Furthermore the '1' in the feature typ will be deleted and a '_' will be added for the boolean categories.

In [13]:
# erasing leading 'u' in column names
merged_data.columns = merged_data.columns.str.lstrip('u')

# renaming columns
merged_data.rename(columns=lambda x: x.replace('ist', 'ist_') if x.startswith('ist') else x, inplace=True)
merged_data.rename(columns={
    'typ1': 'typ',
    'lichtverh': 'licht' 
    }, inplace=True)

merged_data.columns


Index(['land', 'regbez', 'kreis', 'gemeinde', 'jahr', 'monat', 'stunde',
       'wochentag', 'kategorie', 'art', 'typ', 'licht', 'strzustand',
       'ist_rad', 'ist_pkw', 'ist_fuss', 'ist_krad', 'ist_gkfz',
       'ist_sonstige', 'xgcswgs84', 'ygcswgs84'],
      dtype='object')

### Missing Values
Here we check the dataset for missing values.

In [14]:
# check for missing values
merged_data.isnull().sum()

land                 0
regbez               0
kreis                0
gemeinde             0
jahr                 0
monat                0
stunde               0
wochentag            0
kategorie            0
art                  0
typ                  0
licht                0
strzustand           0
ist_rad              0
ist_pkw              0
ist_fuss             0
ist_krad             0
ist_gkfz        195214
ist_sonstige         0
xgcswgs84            0
ygcswgs84            0
dtype: int64

As we can see the data set has almost no null values except for the binary variable 'ist_gkfz'. This will be investigated further and the data imputed where appropriate.

In [15]:
# looking for missing values in 'ist_gkfz'
missing_data = merged_data[merged_data['ist_gkfz'].isnull()]
missing_data.sample(10)

,land,regbez,kreis,gemeinde,jahr,monat,stunde,wochentag,kategorie,art,...,licht,strzustand,ist_rad,ist_pkw,ist_fuss,ist_krad,ist_gkfz,ist_sonstige,xgcswgs84,ygcswgs84
239092,9,1,89,134,2017,7,18,4,3,4,...,0,1,0,1,0,0,NaN,0,12.634336,48.026368
325650,9,4,61,0,2017,5,14,7,3,4,...,0,0,0,1,0,0,NaN,0,10.882665,49.901489
187347,6,4,31,13,2017,8,16,2,2,2,...,0,0,0,1,0,0,NaN,0,8.418425,49.625350
231184,9,6,71,111,2017,5,7,3,3,1,...,0,0,0,1,0,0,NaN,0,9.084743,50.098975
239530,9,7,77,112,2017,7,6,5,3,9,...,0,0,0,1,0,0,NaN,0,10.641603,47.828610
214425,14,5,23,320,2017,5,14,5,3,1,...,0,0,0,1,0,0,NaN,0,12.138438,50.486874
284953,3,4,62,3,2017,9,10,6,3,1,...,0,0,0,1,0,0,NaN,0,7.612477,53.640775
309088,8,2,22,0,2017,11,16,2,3,5,...,0,0,0,1,0,0,NaN,0,8.507881,49.512295
199574,8,2,21,0,2017,6,11,5,3,8,...,0,0,0,1,0,0,NaN,0,8.634315,49.369448
286310,3,3,53,5,2017,10,8,4,3,0,...,0,0,0,0,0,1,NaN,0,9.882314,53.321106


In [16]:
# looking for missing values in 'ist_gkfz' in year 2017
missing_data_2017 = merged_data[merged_data['ist_gkfz'].isnull() & (merged_data['jahr'] == 2017)]
all_null_from_2017 = len(missing_data_2017) == len(missing_data)
not_null_from_2017 = merged_data[merged_data['ist_gkfz'].notnull() & merged_data['jahr'] == 2017].any().any()

print("All the null values for feature is_gkfz are from year 2017: ", all_null_from_2017)
print("There are notnull values for ist_gkfz in year 2017: ", not_null_from_2017)



All the null values for feature is_gkfz are from year 2017:  True
There are notnull values for ist_gkfz in year 2017:  False


Since all null values are only for the feature is_gkfz from 2017 and there are no notnull values for this feature in this year, it can be concluded that no data was collected for is_gkfz in this year.

In the following we will impute the missing values. We will check for patterns and correlation with other features. So we can train a logistic regression model, that imputes our missing values for the year 2017. 

In [17]:
# Calculate the correlation matrix
correlation_matrix = merged_data.corr()

# Select the features with high correlation with is_gkfz
relevant_features = correlation_matrix['ist_gkfz'].abs().sort_values(ascending=False).index[1:]

# map correlation to each feature
correlation_to_ist_gkfz = {feature: correlation_matrix.loc['ist_gkfz', feature] for feature in relevant_features}


# Print the relevant features
print(correlation_to_ist_gkfz)


{'ist_rad': -0.10733862747940964, 'typ': 0.09456375198947359, 'stunde': -0.07848527764753568, 'ist_krad': -0.057456393171918876, 'ist_pkw': -0.057187447091856254, 'kategorie': -0.0519069042703419, 'ist_fuss': -0.046028361121884936, 'art': -0.0393140192823976, 'kreis': 0.02431316433109911, 'licht': -0.019836170951205146, 'gemeinde': 0.018754307961221503, 'wochentag': -0.013051362686227398, 'monat': -0.00970261731071049, 'jahr': -0.00842786096139455, 'ygcswgs84': -0.006115356146857279, 'land': 0.005696697403717505, 'regbez': 0.005381606881152162, 'ist_sonstige': -0.00532538672720455, 'strzustand': 0.00337820718997781, 'xgcswgs84': 0.0032319654037303116}


In [19]:
imputation_data = merged_data.copy()

# delete data with missing values
imputation_data.dropna(inplace=True)

#defining the features and target
features = imputation_data.drop(['ist_gkfz', 'jahr', 'monat', 'land', 'regbez','xgcswgs84', 'ygcswgs84'],  axis=1)
target = imputation_data['ist_gkfz']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create a scaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform
X_train = scaler.fit_transform(X_train)

# Transform the test data
X_test = scaler.transform(X_test)

# Create a logistic regression model
log_reg = LogisticRegression(max_iter=1000)

# Fit the model
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [22]:
# Select the rows with missing values for ist_gkfz
missing_data = merged_data[merged_data['ist_gkfz'].isnull()]

# Extract the features for imputation
imputation_features = missing_data.drop(['ist_gkfz', 'jahr', 'monat', 'land', 'regbez', 'xgcswgs84', 'ygcswgs84'], axis=1)

# Scale the features using the previously created scaler
imputation_features_scaled = scaler.transform(imputation_features)

# Impute the missing values using the logistic regression model
imputed_values = log_reg.predict(imputation_features_scaled)

# Assign the imputed values to the missing rows in the merged_data dataframe
merged_data.loc[merged_data['ist_gkfz'].isnull(), 'ist_gkfz'] = imputed_values

# Check if there are any remaining missing values
merged_data.isnull().sum()


land            0
regbez          0
kreis           0
gemeinde        0
jahr            0
monat           0
stunde          0
wochentag       0
kategorie       0
art             0
typ             0
licht           0
strzustand      0
ist_rad         0
ist_pkw         0
ist_fuss        0
ist_krad        0
ist_gkfz        0
ist_sonstige    0
xgcswgs84       0
ygcswgs84       0
dtype: int64

### Data Aggregation
Now we focus on data aggregation to facilitate later analysis. Specifically, we aggregate the 'Amtliche Gemeinde Schlüssel' (AGS), which is a key identifier for cities in the dataset. This aggregation is essential for streamlining searches and analysis related to specific cities or municipalities. The AGS is created from Land, Regierungsbezirk, Kreis und Gemeinde.

In [23]:
# aggregating ags from given row 
def create_ags(row):
    land = f"{int(row['land']):02d}"  
    regbez = f"{int(row['regbez']):01d}"
    kreis = f"{int(row['kreis']):02d}"  
    gemeinde = f"{int(row['gemeinde']):03d}"  
    return land + regbez + kreis + gemeinde

merged_data['ags'] = merged_data.apply(create_ags, axis=1).astype(str)

# check if ags is created correctly
print(merged_data[['land', 'regbez', 'kreis', 'gemeinde', 'ags']].sample(20))


         land  regbez  kreis  gemeinde       ags
437613      9       1     86       158  09186158
269956      1       0     53        32  01053032
1295343    15       0     88       265  15088265
247424      9       1     86       128  09186128
83378       9       5     74       147  09574147
358859      3       4     57        14  03457014
1299667     1       0     60        63  01060063
360587      3       4      5         0  03405000
478348      3       2     41         1  03241001
1519839     9       1     62         0  09162000
1462930     3       4      4         0  03404000
422773      9       5     65         0  09565000
1089672     5       3     34         2  05334002
303533      8       4     25        33  08425033
1535359    11       0      8         8  11008008
559470      1       0     58       135  01058135
1052004    14       7     30       110  14730110
1461555     3       2     55        23  03255023
942673     14       6     26       110  14626110
435519      9       

As a final step we save the csv so we can access it in other notebooks.

In [24]:
merged_data.to_csv("C:\\Projekte\\TDS\\TDS2324-TrafficAccidents\\Data\\TrafficAccidentData\\all_16_22.csv")